In [1]:
from datasets import Dataset, DatasetDict, Image
from util import TLDataset, readSetFromFile

For the full dataset:

In [2]:
# train_data = TLDataset(
#     "../learning/", "imagery/", "masks/", "*_corrected.png", 
#     subset="Train", fraction=0.1, seed=1,
# )

# validation_data = TLDataset(
#     "../learning/", "imagery/", "masks/", "*_corrected.png", 
#     subset="Test", fraction=0.1, seed=1,
# )

# image_paths_train = [str(path) for path in train_data.image_names]
# label_paths_train = [str(path) for path in train_data.mask_names]

# image_paths_validation = [str(path) for path in validation_data.image_names]
# label_paths_validation = [str(path) for path in validation_data.mask_names]

For the training dataset:

In [3]:
all_data = TLDataset(
    "../learning/", "imagery/", "masks/", "*_corrected.png", 
    subset="Train", fraction=0, seed=1, # We want to select every image
)

indices = readSetFromFile("../data/train_images.txt")
names = [all_data[index]["name"].replace(".png", "") for index in indices]

cutoff = int(len(names) / 10) # fraction=0.1
train_names = names[cutoff:]
val_names = names[:cutoff]

image_paths_train = [f"../learning/imagery/{name}.png" for name in train_names]
label_paths_train = [f"../learning/masks/{name}_corrected.png" for name in train_names]

image_paths_validation = [f"../learning/imagery/{name}.png" for name in val_names]
label_paths_validation = [f"../learning/masks/{name}_corrected.png" for name in val_names]

print(f"Tiny dataset train: {len(train_names)} images")
print(f"Tiny dataset validation: {len(val_names)} images")

Found and loaded 4382 images with glob *_corrected.png.
Subset of 4382 ground truth segmentation masks marked for Train.
Tiny dataset train: 198 images
Tiny dataset validation: 22 images


In [ ]:
def createDataset(image_paths, label_paths):
    dataset = Dataset.from_dict({"image": sorted(image_paths), "label": sorted(label_paths)})
    dataset = dataset.cast_column("image", Image())
    dataset = dataset.cast_column("label", Image())

    return dataset

train_dataset = createDataset(image_paths_train, label_paths_train)
validation_dataset = createDataset(image_paths_validation, label_paths_validation)

dataset = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
})

# This function assumes you have ran the huggingface-cli login command in a terminal/notebook
dataset.push_to_hub("stodoran/elwha-segmentation")

[stodoran/elwha-segmentation](https://huggingface.co/datasets/stodoran/elwha-segmentation/tree/main)